In [2]:
import pandas as pd
import ast

# Optimización de dataset

Auquí nos centraremos en optimizar los datasets para, obtener nuevos con fin de mejorar el rendimiento de las API

In [4]:
# cargamos los conjuntos de datos que preparamos en el ETL
steam_games = pd.read_csv('../datasets/steam_games.csv')
user_reviews = pd.read_parquet('../datasets/user_reviews.parquet')
users_items = pd.read_parquet('../datasets/users_items.parquet')

In [5]:
steam_games.columns

Index(['genres', 'app_name', 'price', 'item_id', 'developer', 'año'], dtype='object')

In [6]:
users_items.columns

Index(['user_id', 'steam_id', 'item_id', 'horas'], dtype='object')

## PlayTimeGenre

In [9]:
# Convertir la columna 'item_id' a tipo int64 en ambos DataFrames
users_items['item_id'] = users_items['item_id'].astype('int64')
steam_games['item_id'] = steam_games['item_id'].astype('int64')

merge = pd.merge(users_items, steam_games[['item_id', 'año', 'genres']], on='item_id', how='inner')

# Ahora haremos un groupby por genero y año de lanzamiento
merge = merge.groupby(['año', 'genres'])['horas'].sum().reset_index()

#Convertiremos la columna genres a lista
merge['genres'] = merge['genres'].apply(ast.literal_eval)

#Pasamos las listas de genres a columnas
merge = merge.explode('genres').reset_index(drop=True)

merge

,año,genres,horas
0,1983,Action,57.883333
1,1983,Adventure,57.883333
2,1983,Casual,57.883333
3,1983,FMV,57.883333
4,1983,Quick-Time Events,57.883333
...,...,...,...
69044,2018,Early Access,1.700000
69045,2018,Platformer,1.700000
69046,2018,Metroidvania,1.700000
69047,2018,RPG,63.350000


In [10]:
# Encontrar el género con más horas jugadas por año 
genre = merge.loc[merge.groupby(['año', 'genres'])['horas'].idxmax()]

# Ordenar el DataFrame resultante
genre = genre.sort_values(by=['genres', 'horas', 'año'], ascending=[True, False, True])

# Guardar el resultado en un archivo CSV
genre.to_csv('../datasets/año_genres.csv', index=False)

## UserForGenre

para esta consulta vamos a trabajar solo con algunas columnas del dataframe steam_games, por lo cual lo guardamos en un nuevo dataframe

In [23]:
user_genre = steam_games.drop(['app_name', 'price', 'developer'], axis=1)

In [24]:
# Realizar la combinación de los DataFrames users_items y user_genre
merge_2 = pd.merge(users_items, user_genre, left_on='item_id', right_on='item_id', how='right')

merge_2.columns

Index(['user_id', 'steam_id', 'item_id', 'horas', 'genres', 'año'], dtype='object')

In [25]:
# Eliminar las columnas 'steam_id', 'item_id' de merge_2 y valores nulos
merge_2.drop(['steam_id', 'item_id'], axis=1, inplace=True)
merge_2.dropna(inplace=True)

In [31]:
#Convertiremos la columna genres a lista
# merge_2['genres'] = merge_2['genres'].apply(ast.literal_eval)

#Expandimos las listas
merge_2 = merge_2.explode('genres').reset_index(drop=True)

In [47]:
# Encontrar el jugador con más horas jugadas por género y año
max_hours = (merge_2.loc[merge_2.groupby(['genres', 'año'])['horas'].idxmax()]
             [['genres', 'año', 'user_id', 'horas']]
             .groupby(['genres', 'año', 'user_id'])['horas']
             .sum()
             .reset_index())

# Exportar el resultado a un archivo CSV
max_hours.to_csv('../datasets/genre_user.csv', index=False)


## UserRecommend

In [36]:
user_reviews.columns

Index(['user_id', 'posted', 'item_id', 'recommend', 'sentiment_analysis'], dtype='object')

In [46]:
user_reviews['item_id'] = user_reviews['item_id'].astype('int64')

merge_3 = pd.merge(user_reviews, steam_games, on='item_id', how='left')

# Filtrar las filas que cumplen con las condiciones
result = merge_3.loc[(merge_3['recommend'] == True) & 
                    (merge_3['sentiment_analysis'].isin([1, 2])),
                    ['año', 'app_name']]

merge_3_group = result.groupby(['año', 'app_name']).size().reset_index(name='count')

merge_3_group = merge_3_group.sort_values(by=['año', 'count'], ascending=[False, False])

top3 = merge_3_group.groupby('año').head(3)

top3.to_csv('../datasets/recomennd.csv', index=False)
top3


,año,app_name,count
2609,2017.0,Unturned,900
2603,2017.0,Robocraft,244
2568,2017.0,ARK: Survival Evolved,214
2480,2016.0,Starbound,325
2335,2016.0,Heroes & Generals,156
...,...,...,...
6,1992.0,Word Rescue,1
3,1991.0,Crystal Caves,1
1,1990.0,Commander Keen,3
2,1990.0,LOOM™,1


## UsersWorstDevelopers

In [51]:
merge_4 = pd.merge(user_reviews, steam_games, on='item_id', how='left')

result = merge_4.loc[(merge_4['recommend'] == False) & 
                    (merge_4['sentiment_analysis'] == 0), 
                    ['año', 'developer']]

count = merge_4.groupby(['año', 'developer']).size().reset_index(name='count')

developer_group = count.sort_values(by=['año', 'count'], ascending=[False, False])

worst_developers = developer_group.groupby('año').head(3)

worst_developers.to_csv('../datasets/worst_developers.csv', index=False)
worst_developers

,año,developer,count
2671,2017.0,Smartly Dressed Games,1050
2646,2017.0,Freejam,346
2672,2017.0,"Studio Wildcard,Instinct Games,Efecto Studios,...",311
2302,2016.0,Chucklefish,351
2517,2016.0,Reto-Moto,242
...,...,...,...
6,1992.0,Redwood Games,1
3,1991.0,3D Realms (Apogee Software),1
2,1990.0,id Software,4
1,1990.0,LucasArts,1


## Sentiment analysis

In [57]:
merge_5 = pd.merge(steam_games, user_reviews, on='item_id', how='inner')

# Agrupar por desarrolladora y sentimiento, y contar las ocurrencias
merge_5 = merge_5.groupby(['developer', 'sentiment_analysis']).size().reset_index(name='count')

# Pivoteamos para obtener tabla de frecuencias de sentimientos
merge_5 = pd.crosstab(index=merge_5['developer'], columns=merge_5['sentiment_analysis']).reset_index()

# Cambiamos nombres de columna
merge_5.columns = ['developer', 'Negative', 'Neutral', 'Positive']


# Convertimos a tipo int
merge_5[['Negative', 'Neutral', 'Positive']] = merge_5[['Negative', 'Neutral', 'Positive']].astype(int)

# Exportamos
merge_5.to_csv('../datasets/sentiment_analysis.csv', index=False)
